# Huomiomekanismit ja transformerit

Yksi toistoverkkojen suurimmista heikkouksista on, että kaikki sanat jaksossa vaikuttavat tulokseen samalla tavalla. Tämä johtaa suboptimaaliseen suorituskykyyn tavanomaisilla LSTM-enkooderi-dekooderi-malleilla sekvenssistä sekvenssiin -tehtävissä, kuten nimettyjen entiteettien tunnistuksessa ja konekäännöksessä. Todellisuudessa tietyillä sanoilla syötejaksossa on usein suurempi vaikutus peräkkäisiin tulosteisiin kuin toisilla.

Ajatellaan sekvenssistä sekvenssiin -mallia, kuten konekäännöstä. Se toteutetaan kahdella toistoverkolla, joissa yksi verkko (**enkooderi**) tiivistää syötejakson piilotilaan ja toinen verkko, **dekooderi**, purkaa tämän piilotilan käännetyksi tulokseksi. Tämän lähestymistavan ongelmana on, että verkon lopullisen tilan on vaikea muistaa lauseen alkua, mikä heikentää mallin laatua pitkissä lauseissa.

**Huomiomekanismit** tarjoavat keinon painottaa kunkin syötevektorin kontekstuaalista vaikutusta RNN:n jokaisessa tulosennusteessa. Tämä toteutetaan luomalla oikopolkuja syötteen RNN:n välitilojen ja tulos-RNN:n välille. Näin ollen, kun tuotetaan tulossymbolia $y_t$, otamme huomioon kaikki syötteen piilotilat $h_i$, eri painokertoimilla $\alpha_{t,i}$. 

![Kuva, joka esittää enkooderi/dekooderi-mallin additiivisella huomiokerroksella](../../../../../translated_images/fi/encoder-decoder-attention.7a726296894fb567.webp)
*Enkooderi-dekooderi-malli additiivisella huomiomekanismilla [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), lainattu [tästä blogikirjoituksesta](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Huomiomatriisi $\{\alpha_{i,j}\}$ edustaa sitä, kuinka paljon tietyt syötteen sanat vaikuttavat tietyn sanan muodostumiseen tulosjaksossa. Alla on esimerkki tällaisesta matriisista:

![Kuva, joka näyttää esimerkkikohdistuksen RNNsearch-50:llä, otettu Bahdanau - arviz.org](../../../../../translated_images/fi/bahdanau-fig3.09ba2d37f202a6af.webp)

*Kuva otettu [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Kuva 3)*

Huomiomekanismit ovat vastuussa suuresta osasta nykyistä tai lähes nykyistä huipputasoa luonnollisen kielen käsittelyssä. Huomion lisääminen kuitenkin kasvattaa huomattavasti mallin parametrien määrää, mikä johti skaalautumisongelmiin RNN:ien kanssa. Yksi RNN:ien skaalautumisen keskeisistä rajoitteista on, että mallien toistuva luonne tekee koulutuksen eräajosta ja rinnakkaistamisesta haastavaa. RNN:ssä jokainen sekvenssin elementti täytyy käsitellä järjestyksessä, mikä tarkoittaa, että sitä ei voida helposti rinnakkaistaa.

Huomiomekanismien käyttöönotto yhdessä tämän rajoitteen kanssa johti nykyisten huipputason Transformer-mallien luomiseen, joita käytämme tänään, kuten BERT ja OpenGPT3.

## Transformer-mallit

Sen sijaan, että jokaisen edellisen ennusteen konteksti välitettäisiin seuraavaan arviointivaiheeseen, **transformer-mallit** käyttävät **paikkakoodauksia** ja **huomiota** tallentaakseen annetun syötteen kontekstin annetun tekstin ikkunan sisällä. Alla oleva kuva näyttää, kuinka paikkakoodaukset ja huomio voivat tallentaa kontekstin annetun ikkunan sisällä.

![Animoitu GIF, joka näyttää, kuinka arvioinnit suoritetaan transformer-malleissa.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif) 

Koska jokainen syötepaikka kartoitetaan itsenäisesti jokaiseen tulospaikkaan, transformerit voivat rinnakkaistaa paremmin kuin RNN:t, mikä mahdollistaa paljon suuremmat ja ilmaisukykyisemmät kielimallit. Jokainen huomiopää voidaan käyttää oppimaan erilaisia suhteita sanojen välillä, mikä parantaa luonnollisen kielen käsittelyn tehtäviä.

## Yksinkertaisen Transformer-mallin rakentaminen

Keras ei sisällä sisäänrakennettua Transformer-kerrosta, mutta voimme rakentaa oman. Kuten aiemmin, keskitymme AG News -datan tekstiluokitteluun, mutta on syytä mainita, että Transformer-mallit saavuttavat parhaat tulokset vaikeammissa NLP-tehtävissä.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

Uudet kerrokset Kerasissa tulisi aliluokittaa `Layer`-luokka ja toteuttaa `call`-metodi. Aloitetaan **Positional Embedding** -kerroksella. Käytämme [joitakin koodinpätkiä virallisesta Keras-dokumentaatiosta](https://keras.io/examples/nlp/text_classification_with_transformer/). Oletamme, että täytämme kaikki syötesequencet pituuteen `maxlen`.


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

Tämä kerros koostuu kahdesta `Embedding`-kerroksesta: yksi tokenien upottamiseen (kuten aiemmin on käsitelty) ja toinen tokenien sijaintien upottamiseen. Tokenien sijainnit luodaan luonnollisten lukujen sekvenssinä välillä 0 ja `maxlen` käyttäen `tf.range`, ja ne syötetään upotuskerroksen läpi. Kaksi syntynyttä upotusvektoria yhdistetään yhteen, jolloin saadaan sijaintiin perustuva upotettu esitys syötteestä, jonka muoto on `maxlen`$\times$`embed_dim`.

Nyt toteutetaan transformer-lohko. Se ottaa syötteenä aiemmin määritellyn upotuskerroksen tuottaman ulostulon:


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

Nyt olemme valmiita määrittämään täydellisen transformer-mallin:


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## BERT Transformer -mallit

**BERT** (Bidirectional Encoder Representations from Transformers) on erittäin suuri monikerroksinen transformer-verkko, jossa on 12 kerrosta *BERT-base*-mallissa ja 24 kerrosta *BERT-large*-mallissa. Malli esikoulutetaan ensin suurella tekstiaineistolla (WikiPedia + kirjat) käyttämällä valvomatonta oppimista (ennustamalla peitettyjä sanoja lauseessa). Esikoulutuksen aikana malli omaksuu merkittävän määrän kielellistä ymmärrystä, jota voidaan hyödyntää muiden aineistojen kanssa hienosäädön avulla. Tätä prosessia kutsutaan **siirto-oppimiseksi**.

![kuva osoitteesta http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/fi/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Transformer-arkkitehtuureista on monia variaatioita, kuten BERT, DistilBERT, BigBird, OpenGPT3 ja muita, joita voidaan hienosäätää.

Katsotaanpa, kuinka voimme käyttää esikoulutettua BERT-mallia perinteisen sekvenssiluokitusongelman ratkaisemiseen. Hyödynnämme ideaa ja hieman koodia [virallisesta dokumentaatiosta](https://www.tensorflow.org/text/tutorials/classify_text_with_bert).

Esikoulutettujen mallien lataamiseen käytämme **Tensorflow hub** -kirjastoa. Aloitetaan lataamalla BERT-spesifinen vektoroija:


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

On tärkeää käyttää samaa vektoroijaa, jolla alkuperäinen verkko on koulutettu. Lisäksi BERT-vektoroija palauttaa kolme komponenttia:
* `input_word_ids`, joka on syötelauseen token-numeroiden sekvenssi
* `input_mask`, joka näyttää, mikä osa sekvenssistä sisältää varsinaisen syötteen ja mikä osa on täytettä. Se muistuttaa `Masking`-kerroksen tuottamaa maskia
* `input_type_ids` käytetään kielimallinnustehtävissä ja mahdollistaa kahden syötelauseen määrittämisen yhdeksi sekvenssiksi.

Sen jälkeen voimme luoda BERT-ominaisuuksien erottimen:


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


Joten, BERT-kerros palauttaa joukon hyödyllisiä tuloksia:
* `pooled_output` on tulos, joka saadaan keskiarvoistamalla kaikki sekvenssin tokenit. Voit ajatella sen älykkäänä semanttisena upotuksena koko verkosta. Se vastaa `GlobalAveragePooling1D`-kerroksen tulosta aiemmassa mallissamme.
* `sequence_output` on viimeisen transformer-kerroksen tulos (vastaa `TransformerBlock`-kerroksen tulosta yllä olevassa mallissamme).
* `encoder_outputs` ovat kaikkien transformer-kerrosten tulokset. Koska olemme ladanneet 4-kerroksisen BERT-mallin (kuten nimestä, joka sisältää `4_H`, voi päätellä), siinä on 4 tensoria. Viimeinen niistä on sama kuin `sequence_output`.

Nyt määrittelemme end-to-end-luokittelumallin. Käytämme *funktionaalista mallin määrittelyä*, jossa määritämme mallin syötteen ja annamme sitten joukon lausekkeita sen tuloksen laskemiseksi. Teemme myös BERT-mallin painot ei-opetettaviksi ja opetamme vain lopullisen luokittelijan:


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


Vaikka koulutettavia parametreja on vähän, prosessi on melko hidas, koska BERT-ominaisuuksien uuttaja on laskennallisesti raskas. Näyttää siltä, ettemme onnistuneet saavuttamaan kohtuullista tarkkuutta, joko koulutuksen puutteen tai mallin parametrien vähyyden vuoksi.

Yritetään avata BERT-painot ja kouluttaa sitä myös. Tämä vaatii erittäin pienen oppimisnopeuden sekä tarkemman koulutusstrategian, jossa käytetään **warmupia** ja **AdamW**-optimointialgoritmia. Käytämme `tf-models-official`-pakettia luodaksemme optimointialgoritmin:


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

Kuten huomaat, koulutus etenee melko hitaasti – mutta saatat haluta kokeilla ja kouluttaa mallia muutaman epochin ajan (5–10) ja katsoa, voitko saada parhaan tuloksen verrattuna aiemmin käyttämiimme lähestymistapoihin.

## Huggingface Transformers -kirjasto

Toinen hyvin yleinen (ja hieman yksinkertaisempi) tapa käyttää Transformer-malleja on [HuggingFace-paketti](https://github.com/huggingface/), joka tarjoaa yksinkertaisia rakennuspalikoita erilaisiin NLP-tehtäviin. Se on saatavilla sekä Tensorflow- että PyTorch-ympäristöihin, jotka ovat muita erittäin suosittuja neuroverkkojen kehysalustoja.

> **Huom**: Jos et ole kiinnostunut näkemään, miten Transformers-kirjasto toimii – voit hypätä tämän muistikirjan loppuun, koska et tule näkemään mitään olennaisesti erilaista kuin mitä olemme tehneet aiemmin. Toistamme samat vaiheet BERT-mallin kouluttamisessa käyttäen eri kirjastoa ja huomattavasti suurempaa mallia. Näin ollen prosessi sisältää melko pitkää koulutusta, joten saatat haluta vain selata koodin läpi.

Katsotaanpa, miten ongelmamme voidaan ratkaista käyttämällä [Huggingface Transformers](http://huggingface.co).


Ensimmäinen asia, joka meidän täytyy tehdä, on valita käytettävä malli. Huggingfacella on sisäänrakennettujen mallien lisäksi [verkkopohjainen mallivarasto](https://huggingface.co/models), josta löydät paljon lisää yhteisön valmiiksi kouluttamia malleja. Kaikki nämä mallit voidaan ladata ja käyttää pelkästään antamalla mallin nimi. Kaikki tarvittavat binääritiedostot mallia varten ladataan automaattisesti.

Joissain tilanteissa sinun täytyy ladata omia mallejasi. Tällöin voit määrittää hakemiston, joka sisältää kaikki tarvittavat tiedostot, kuten tokenisoijan parametrit, `config.json`-tiedoston mallin parametreilla, binääripainot jne.

Mallin nimen perusteella voimme luoda sekä mallin että tokenisoijan. Aloitetaan tokenisoijasta:


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

`tokenizer`-objekti sisältää `encode`-funktion, jota voidaan käyttää suoraan tekstin koodaamiseen:


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Voimme myös käyttää tokenisoijaa koodataksemme sekvenssin tavalla, joka sopii mallille välittämiseen, eli sisältäen `token_ids`, `input_mask` kentät jne. Voimme myös määrittää, että haluamme Tensorflow-tensoreita antamalla `return_tensors='tf'` argumentin:


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

Tässä tapauksessa käytämme valmiiksi koulutettua BERT-mallia nimeltä `bert-base-uncased`. *Uncased* tarkoittaa, että malli ei erota isoja ja pieniä kirjaimia.

Kun mallia koulutetaan, meidän täytyy antaa syötteeksi tokenisoitu sekvenssi, joten suunnittelemme datankäsittelyputken. Koska `tokenizer.encode` on Python-funktio, käytämme samaa lähestymistapaa kuin edellisessä osiossa kutsumalla sitä `py_function`-toiminnolla:


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

Nyt voimme ladata varsinaisen mallin käyttämällä `BertForSequenceClassification`-pakettia. Tämä varmistaa, että mallillamme on jo tarvittava luokitteluun tarkoitettu arkkitehtuuri, mukaan lukien lopullinen luokitin. Näet varoitusviestin, jossa todetaan, että lopullisen luokittimen painot eivät ole alustettuja ja malli vaatii esikoulutusta - tämä on täysin normaalia, sillä juuri sitä olemme tekemässä!


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


Kuten näet `summary()`-funktiosta, malli sisältää lähes 110 miljoonaa parametria! Oletettavasti, jos haluamme yksinkertaisen luokittelutehtävän suhteellisen pienellä datalla, emme halua kouluttaa BERT-peruskerrosta:


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


Nyt voimme aloittaa mallin koulutuksen!

> **Huomio**: Täysimittaisen BERT-mallin kouluttaminen voi olla erittäin aikaa vievää! Siksi koulutamme sitä vain ensimmäiset 32 erää. Tämä on vain esimerkki siitä, miten mallin koulutus asetetaan. Jos haluat kokeilla täysimittaista koulutusta - poista vain `steps_per_epoch` ja `validation_steps` -parametrit, ja varaudu odottamaan!


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


Jos lisäät iteraatioiden määrää ja odotat tarpeeksi kauan sekä koulutat useita epookkeja, voit odottaa, että BERT-luokittelu antaa meille parhaan tarkkuuden! Tämä johtuu siitä, että BERT ymmärtää jo varsin hyvin kielen rakenteen, ja meidän tarvitsee vain hienosäätää lopullista luokittelijaa. Kuitenkin, koska BERT on suuri malli, koko koulutusprosessi vie paljon aikaa ja vaatii merkittävää laskentatehoa! (GPU, ja mieluiten useampi kuin yksi).

> **Huom:** Esimerkissämme olemme käyttäneet yhtä pienimmistä valmiiksi koulutetuista BERT-malleista. On olemassa suurempia malleja, jotka todennäköisesti tuottavat parempia tuloksia.


## Yhteenveto

Tässä osiossa olemme tutustuneet hyvin uusiin mallirakenteisiin, jotka perustuvat **transformereihin**. Olemme soveltaneet niitä tekstiluokittelutehtäväämme, mutta vastaavasti BERT-malleja voidaan käyttää esimerkiksi entiteettien tunnistamiseen, kysymysten vastaamiseen ja muihin NLP-tehtäviin.

Transformer-mallit edustavat NLP:n tämänhetkistä huipputasoa, ja useimmissa tapauksissa niiden tulisi olla ensimmäinen ratkaisu, jota kokeilet, kun toteutat räätälöityjä NLP-ratkaisuja. Kuitenkin, jos haluat rakentaa kehittyneitä neuroverkkomalleja, on äärimmäisen tärkeää ymmärtää tämän moduulin yhteydessä käsitellyt perusperiaatteet toistuvista neuroverkoista.



---

**Vastuuvapauslauseke**:  
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Vaikka pyrimme tarkkuuteen, huomioithan, että automaattiset käännökset voivat sisältää virheitä tai epätarkkuuksia. Alkuperäistä asiakirjaa sen alkuperäisellä kielellä tulisi pitää ensisijaisena lähteenä. Kriittisen tiedon osalta suositellaan ammattimaista ihmiskäännöstä. Emme ole vastuussa väärinkäsityksistä tai virhetulkinnoista, jotka johtuvat tämän käännöksen käytöstä.
